# Words by Numbers: Exploring Vector Similarity 
This notebook accompanies our readings on AI, to help explore words of interest as "vector embeddings". As we are learning from our readings, in the making of a large language model, they consist of calculations about meaningful units of language (words / tokenized particles) and they calculate relationships based on mathematical vectors. 

In this digital humanities orientation to language models and word embeddings, we'll reach for the number values ("vector word embedding values") that language models apply to word-tokens when they read and generate them.

*My first version of this notebook is [this Python file](https://github.com/newtfire/textAnalysis-Hub/blob/main/Class-Examples/Python/readFileCollections-examples/readingFileCollection.py) also stored in our textAnalysis-Hub Python collection.

## Pay attention to an interesting word: A little lab test of LLMs
We'll just do this as a test case based on some text file outputs from any generative AI. (You can also adapt this script to explore your own text corpora for your projects.) For our demonstration, we'll run a little experiment on generative AI chatbots (ChatGPT, Claude, Gemini, etc). We can give a couple of these bots the same prompt, and save the outputs in a directory. Then we'll run this notebook over them.

When you review outputs to your prompts, do you see a **word or idea** that's sometimes repeated or that seems an interesting basis for comparison across the files? Take your cue from this: Choose a word of interest that you think is worth "paying attention to" as a basis for comparing the text outputs of the AI (or the texts in your collection). In this notebook, we'll use it as a basis for comparing all the files: **how often do they use a similar word**, that is, a "similar" word based on a language model's calculations of similarity? 

For our experiment, we'll work with spaCy's freely available large language model, to access its vector database on words and return calculations. Python can "crunch the numbers" to show us which prompts are using words that are mapped to a nearby "vector space" in spaCy's language model. This Python script can help show us how our texts compare to each other based on how often they draw from words that share similar number values. 

For this experiment we'll be accessing the **cosine similarity** values that LLMs reach for in calculating a response to a prompt. The only difference is, we're using spaCy's embedding dictionary as our basis for exploring. (Our little experiment is simply using spaCy's vector embeddings as our similarity measuring instrument. To take this further, we could try to get "under the hood" of each LLM to access its distinctive vector database.)

## Installs and imports
* spacy
* spacy's large language model
* os for handling filepaths


In [ ]:
import spacy
import os

nlp = spacy.load('en_core_web_lg')
# COMMENT OUT THE spacy.load line after the first time you load your model!
# If the large one is too big, you can use the medium:
# nlp = spacy.load('en_core_web_md')
# en_core_web_md: size: 34 MB )
# en_core_web_lg (Lots of data here, size: 400 MB.) Note that the LARGEST one will have the most data and probably be the most reliable.


### Identify the directory to read
So, you have a collection of text files that you want to compare, and you've stored them in a directory(folder). 
Which directory do you want the Python script to read? Define it in relation to where you saved your Python file.

In [ ]:
collection = 'textCollection'
# This is a folder saved in the same directory with this Python notebook.

In [ ]:
# "Smoke test": Open the directory and make sure we can access the files inside. 
# Here's where we use os.
def readTextFiles(filepath):
    with open(filepath, 'r', encoding='utf8') as f:
    # ebb: add that utf8 encoding argument to the open() function to ensure that reading works on everyone's systems
    # this all succeeds if you see the text of your files printed in the console.
        readFile = f.read()
        # print(readFile)
        stringFile = str(readFile)
        lengthFile = len(readFile)
        # (Literally, we'll just count the number of characters in this.)
        print(lengthFile)

for file in os.listdir(collection):
    if file.endswith(".txt"):
        filepath = f"{collection}/{file}"
        print(filepath)
        readTextFiles(filepath)

### Apply spaCy's language model 

* First we'll have spaCy tokenize and "read" the files, just using its default tokenizer.
* Then we'll choose our **word of interest** that we'll use as a basis for exploring similarity.
* We'll then create a **dictionary of highly similar values** to that word, drawn from spaCy's vector database.

The similarity value is called "cosine similarity" and it varies between 0 and 1, with 1 being closest to identical.
Try adjusting the "similarity gage" by tweaking the number in the second if statement: 

```py
            if wordOfInterest.similarity(token) > .3:
```

How does changing this value affect the results you see?

 

In [ ]:
def readTextFiles(filepath):
    with open(filepath, 'r', encoding='utf8') as f:
        readFile = f.read()
        # print(readFile)
        stringFile = str(readFile)

        tokens = nlp(stringFile)
        # playing with vectors here
        vectors = tokens.vector
        # Want to "see" some vector information? Uncomment the  next line:
        # print(vectors)

        wordOfInterest = nlp(u'panic')

         # Now, let's open an empty dictionary! We'll fill it up with the for loop just after it.
        # The for-loop goes over each token and gets its values
        highSimilarityDict = {}
        for token in tokens:
            if(token and token.vector_norm):
                if wordOfInterest.similarity(token) > .4:
                # ^^^^ our "similarity gage" ^^^^
                    highSimilarityDict[token] = wordOfInterest.similarity(token)
                    # The line above creates the structure for each entry in my dictionary.
                        # print(token.text, "about this much similar to", wordOfInterest, ": ", wordOfInterest.similarity(token))
        print(f'This is a dictionary of words most similar to the word "{wordOfInterest.text}" in "{file}".')
        print(highSimilarityDict)
        print('\n')

        

for file in os.listdir(collection):
    if file.endswith(".txt"):
        filepath = f"{collection}/{file}"
        readTextFiles(filepath)

#### "Dedupe" and sort the dictionary
The results above give you duplicate words based on how often they appear in the text. That's information we might want to count, but still just have the word appear once. We might also want to sort the values from most to least similar to our word of interest.

So we'll try to do these things in the next cell. The **Counter** function from the collections library will be super helpful here because it takes distinct values AND counts how often a value appears in the text. 


In [ ]:
from collections import Counter

def readTextFiles(filepath):
    with open(filepath, 'r', encoding='utf8') as f:
        readFile = f.read()
        # print(readFile)
        stringFile = str(readFile)

        tokens = nlp(stringFile)
        # playing with vectors here
        vectors = tokens.vector
        # Want to "see" some vector information? Uncomment the  next line:
        # print(vectors)

        wordOfInterest = nlp(u'panic')

        # Our structures for storing similarity data:
        # This time we'll add counts, 
        # and we'll use Python's sorted() function to sort by similarity values from high (close to 1) to low (close to 0):
        highSimilarityDict = {}
        sorted_similarity = sorted(highSimilarityDict.items(), key=lambda item: item[1], reverse=True)
        wordCounts = Counter()
        
        for token in tokens:
            if(token and token.vector_norm):
                if wordOfInterest.similarity(token) > .4:
                    wordCounts[token.text] += 1
                # ^^^^ our "similarity gage" ^^^^
                    highSimilarityDict[token] = wordOfInterest.similarity(token)
                    # The line above creates the structure for each entry in my dictionary.
                        # print(token.text, "about this much similar to", wordOfInterest, ": ", wordOfInterest.similarity(token))
        # Smoke test for the Counter(): 
        # for word, count in list(wordCounts.items())[:5]:
        #    print(f"'{word}': {count}")
        print(f'This is a dictionary of words most similar to the word "{wordOfInterest.text}" in "{file}".')
        for word, similarity in highSimilarityDict.items():
            count = wordCounts[word.text]
            print(f"{word}: similarity={similarity:.3f}, count={count}")
            # The `:3f` above basically just reduces the decimal places to three (as in 3 floating points). 
            # print(f"{word}: similarity={similarity}, count={count}")
        print('\n')
 

for file in os.listdir(collection):
    if file.endswith(".txt"):
        filepath = f"{collection}/{file}"
        readTextFiles(filepath)


## Your Turn! 

Adapt this code in your own Python file or notebook to explore similarity values based on your own collection of text files. Is this of interest in your project?